In [2]:
# Floyd Warshall algorithm  O(n^3)

# Let's say P[i,j,k] is shortest distance solution from i to j, with k ∈ {1,2..k} allowed to be intermidiary point
# If k is not inside intermidary points for i->j
# P[i,j,k] must == P[i,j,k-1]
# Elif k is a intermediary point
# P[i,j,k] must == P[i,k,k-1] + P[j,k,k-1]

# P[i,i, n] must = 0
# when k = 0, 
# if i==j, P[i,j,k] = 0
# if i, j connected, P[i,j,k] = Cij(distance bewteen i and j)
# else P[i,j,k] = positive inf

# if graph has a negative cycle, will get P[i, i, n] < 0 for at least one i at the end of algorithm

In [3]:
import re
from collections import defaultdict

def file_to_graph(filename):  # head
    with open(filename) as f:
        content = f.readlines()    
    pattern = re.compile('\d+')
    matches = [re.findall(pattern, e) for e in content]  
    # first line is number of nodes and edges
    num_of_nodes, num_of_edges = [int(x) for x in matches[0]]
    integers = [[int(x) for x in n] for n in matches[1:]]
    graph = defaultdict(list)
    for x in integers:
        graph[x[0]].append({'head': x[1], 'length': x[2]})
    return graph

def file_to_graph_(filename):  # tail
    with open(filename) as f:
        content = f.readlines()    
    pattern = re.compile('\d+')
    matches = [re.findall(pattern, e) for e in content]  
    # first line is number of nodes and edges
    num_of_nodes, num_of_edges = [int(x) for x in matches[0]]
    integers = [[int(x) for x in n] for n in matches[1:]]
    graph = defaultdict(list)
    for x in integers:
        graph[x[1]].append({'tail': x[0], 'length': x[2]})
    return graph

In [38]:
graph = file_to_graph('datasets/bellman_ford_test.txt')
graph

defaultdict(list,
            {1: [{'head': 2, 'length': 2}, {'head': 5, 'length': 3}],
             2: [{'head': 4, 'length': 2}],
             3: [{'head': 1, 'length': 1}],
             4: [{'head': 1, 'length': 4},
              {'head': 3, 'length': 1},
              {'head': 5, 'length': 2}],
             5: [{'head': 3, 'length': 1}]})

In [7]:
import numpy as np
import pandas as pd

def floyd(graph):
    nodes = list(graph.keys())
    n = len(nodes)
    panel = pd.Panel(np.full((n+1, n, n), np.inf), 
                     items=range(0, n+1),
                     major_axis=range(1, n+1),
                     minor_axis=range(1, n+1))
    df = panel.to_frame()
    # set values for k = 0
    for i in range(1,n+1):
        df.loc[i,i].loc[0] = 0
        tails = graph[i]
        for t in tails:
            df.loc[i, t['head']].loc[0] = t['length']
    # main logic
    for k in range(1, n+1):
        for i in range(1, n+1):
            for j in range(1, n+1):
                m1 = df.loc[i,j].loc[k-1]
                m2 = df.loc[i,k].loc[k-1] + df.loc[k,j].loc[k-1]
                df.loc[i,j].loc[k] = min(m1, m2)
    # get shortest distance and check negative cycle
    output = df[n] # k = n, get shortest distance for each pair of i,j
    for i in range(1, n+1):
        if output[i][i] < 0:
            raise ValueError('Negative cycle found!')
            
    return output # to get the shortest_dist from i to j, do output[i][j] or output.loc[i, j]

In [9]:
x = floyd(graph)
x[1,4]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


4.0

In [25]:
from collections import defaultdict

def convert_graph(graph):
    out = defaultdict(dict)
    for k, v in graph.items():
        for x in v:
            out[k][x['head']] = x['length']
    return out

def convert_graph_list(graph):
    out = []
    for k, v in graph.items():
        for x in v:
            out.append((k, x['head'], x['length']))
    return out

gg = convert_graph(graph)
gg

defaultdict(dict,
            {1: {2: 2, 5: 3},
             2: {4: 2},
             3: {1: 1},
             4: {1: 4, 3: 1, 5: 2},
             5: {3: 1}})

In [33]:
# Johnson's algorithm

import networkx as nx
g = nx.DiGraph()
g.add_weighted_edges_from([(1, 2, 2), (1, 5, 3), (2,4, 2), (3, 1, 1), (4, 1, 4),(4,3,1),(4,5,2),(5,3,1)])
paths = nx.johnson(g, weight='weight')
return paths  # shortest paths for each pair 

def calc_length(graph, path):
    output = 0
    for i in range(len(path)-1):
        output += graph[path[i]][path[i+1]]
    return output

In [34]:
calc_length(gg, paths[1][4])  # the shortest path length from 1 to 4

4

In [40]:
g1 = file_to_graph('datasets/g1.txt')
g2 = file_to_graph('datasets/g2.txt')
g3 = file_to_graph('datasets/g3.txt')

In [49]:
# In this assignment you will implement one or more algorithms for the all-pairs shortest-path problem.
# Here are data files describing three graphs:

# The first line indicates the number of vertices and edges, respectively. 
# Each subsequent line describes an edge (the first two numbers are its tail and head, respectively) 
# and its length (the third number). 
# NOTE: some of the edge lengths are negative. 
# NOTE: These graphs may or may not have negative-cost cycles.

# Your task is to compute the "shortest shortest path". Precisely, you must first identify which, 
# if any, of the three graphs have no negative cycles. For each such graph, you should compute 
# all-pairs shortest paths and remember the smallest one 
# (i.e., compute minu,vExistVd(u,v), where d(u,v) denotes the shortest-path distance from u to v).

# If each of the three graphs has a negative-cost cycle, then enter "NULL" in the box below. 
# If exactly one graph has no negative-cost cycles, then enter the length of its shortest shortest 
# path in the box below. If two or more of the graphs have no negative-cost cycles, then enter the 
# smallest of the lengths of their shortest shortest paths in the box below.

# OPTIONAL: You can use whatever algorithm you like to solve this question. If you have extra time, 
# try comparing the performance of different all-pairs shortest-path algorithms!

GRAPH_test_1_FILENAME = "data/tests/all-pairs-shortest-path-graph-test-1.txt" # result: negative cycle
GRAPH_test_2_FILENAME = "data/tests/all-pairs-shortest-path-graph-test-2.txt" # result: negative cycle
GRAPH_test_3_FILENAME = "data/tests/all-pairs-shortest-path-graph-test-3.txt" # result: -3

GRAPH_1_FILENAME = "data/all-pairs-shortest-path-graph-1.txt" # result: negative cycle
GRAPH_2_FILENAME = "data/all-pairs-shortest-path-graph-2.txt" # result: negative cycle
GRAPH_3_FILENAME = "data/all-pairs-shortest-path-graph-3.txt" # result: -19

# graphs = [GRAPH_1_FILENAME, GRAPH_2_FILENAME, GRAPH_3_FILENAME] # result: -19
graphs = [GRAPH_test_1_FILENAME, GRAPH_test_2_FILENAME, GRAPH_test_3_FILENAME] # result: -3


def initialize(graph, source):
    destination = {}
    predecessor = {}
    for vertice in graph:
        destination[vertice] = float('Inf')
        predecessor[vertice] = None
    destination[source] = 0
    return destination, predecessor

def relax(vertice, previous, graph, distance, predecessor):
    if distance[previous] > distance[vertice] + graph[vertice][previous]:
        distance[previous] = distance[vertice] + graph[vertice][previous]
        predecessor[previous] = vertice

def bellman_ford(graph, source):
    global has_cycle

    distance, predecessor = initialize(graph, source)
    for i in range(len(graph)-1):
        for u in graph:
            for v in graph[u]:
                relax(u, v, graph, distance, predecessor)

    # check for negative cycles
    for u in graph:
        for v in graph[u]:
            if distance[v] > distance[u] + graph[u][v]:
                has_cycle = True

    return distance, predecessor

def getSmallest(distances, smallest):
    for idx in distances:
        if not smallest or distances[idx] < smallest:
            smallest = distances[idx]

    return smallest

smallest = False

for g in graphs:

    small = False
    has_cycle = False
    inFile = open(g, 'r') 

    num_vertices = 0
    num_edges = 0
    graph = {}
    vertices = []

    # initializing the graph with an array of edges [tale, head]
    for f in inFile:
        if(num_vertices == 0):
            num_vertices, num_edges = map(int, f.split())
        else: 
            tale, head, length = map(int, f.split())
            if tale not in vertices:
                vertices.append(tale)
            if head not in vertices:
                vertices.append(head)
            if tale not in graph:
                graph[tale] = {}
            graph[tale][head] = length

    vertices = sorted(vertices)

    # adding vertices without outgoing edges
    for vertice in vertices:
        if vertice not in graph:
            graph[vertice] = {}

    for vertice in vertices:
        d, p = bellman_ford(graph, vertice)
        small = getSmallest(d, small)

    if has_cycle:
        continue
    elif not smallest or smallest > small:
        smallest = small


if not smallest:
    print 'result: NULL'
else:
    print 'result: ' + str(smallest)
